#       **1. PRE-PROCESAMIENTO DE IMAGENES SENTINEL - 2 Y VARIABLES EXPLICATIVAS**

Permite el analisis de imagenes satelitales en la nube utilizando Google Earth Engine; permitiendo obtener una imagen Sentinel 2 sin nubes y con índices y variables explicativas para el modelo para el área de interés

## 1.1 Importar librerias y cargar área de interés

In [7]:
import ee
import geemap
import geopandas as gpd

gdf_aoi=gpd.read_file('C:/Users/Equipo/SemDocker/ProyectoProgramacionSIG/Data/AOI/PNNAmacayacu.shp') # Path to the vector file that contains the AOI

ee.Authenticate()

ee.Initialize(project='ee-maperezla')

True

## 1.2 Convertir geometria local a formato Earth Engine

In [9]:
# Convert to Earth Engine geometry
aoi_geom = gdf_aoi.geometry[0].__geo_interface__
aoi_ee = ee.Geometry(aoi_geom)

## 1.3 Definir las bandas y filtrar la colección Sentinel-2

La banda **QA60** contiene bit de control de calidad que incluye información sobre nubes y cirros, realizandose una operación bit a bit 

In [11]:
# Definir las bandas y filtrar la colección Sentinel-2
bandas = ['B2', 'B3', 'B4', 'B5','B6','B8','B8A','B9','B11','B12']

# Función para enmascarar nubes y convertir a reflectancia
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    # Aplicar la máscara, dividir por 10000 y conservar solo las bandas ópticas
    return image.updateMask(mask).divide(10000).select(bandas)

# Filtrar, mapear, calcular mediana y recortar
sentinel2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2023-01-01', '2023-12-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .filterBounds(aoi_ee)
    .map(maskS2clouds)
    .median()
    .clip(aoi_ee)
)

# --- Configuración para Visualización con geemap ---

# Parámetros de visualización para la imagen Sentinel-2 (RGB natural)
# B4 (Red), B3 (Green), B2 (Blue)
sentinel_vis_params = {
    'min': 0,
    'max': 3000, # Ajusta estos valores según la luminosidad deseada
    'bands': ['B4', 'B3', 'B2']
}

# Parámetros de visualización para el AOI (opcional, para resaltar el polígono)
aoi_vis_params = {
    'color': 'FF0000', # Rojo en formato hexadecimal (RGB)
    'width': 2,        # Ancho del borde
    'fillColor': '00000000' # Fondo transparente (ARGB: 00 = alfa totalmente transparente)
}

## 1.4 Crear un mapa interactivo centrado en el área de estudio

In [12]:

# Usamos el centroide del primer polígono del gdf para el centrado inicial del mapa
Map = geemap.Map(center=[gdf_aoi.geometry[0].centroid.y, gdf_aoi.geometry[0].centroid.x], zoom=9)

# Agregar la imagen Sentinel-2 al mapa
Map.addLayer(sentinel2, sentinel_vis_params, 'Sentinel-2 (2023-11-01 to 2023-12-31)')

# Agregar el AOI (FeatureCollection) al mapa
Map.addLayer(aoi_ee, aoi_vis_params, 'Área de Interés (AOI)')

# Añadir control de capas para encender/apagar y ajustar la opacidad
Map.addLayerControl()

# Mostrar el mapa
Map

Map(center=[-3.439523286382691, -70.1442113276866], controls=(WidgetControl(options=['position', 'transparent_…

## 1.5 Extracción de variables explicativas como índices y bandas de interes para el modelo

In [15]:
import sys
sys.path.append('C:/Users/Equipo/SemDocker/ProyectoProgramacionSIG/src_25_07_04/src')
import indices_sentinel2

In [16]:
from indices_sentinel2 import indices_sentinel2

image_indices = indices_sentinel2(sentinel2)

In [17]:
print (image_indices.bandNames().getInfo())

['NDVI', 'EVI', 'BSI', 'RED', 'NIR', 'SWIR1', 'SWIR2']


In [18]:
image_indices_64 = image_indices.toDouble()

## 1.6. Exportar la composción de Sentinel-2

In [19]:
# Export the composite image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=image_indices_64,
    description='Sentinel2_INDEX_Proyecto',
    folder='earthengine',            # Destination folder in Google Drive
    fileNamePrefix='sentinel2_index_proye', # File name without extension
    region=aoi_ee,                   # Area to export
    scale=10,                        # Set a very high limit to allow exporting large images
                                     # Earth Engine requires this to avoid exporting by mistake very large rasters
                                     # The default limit is 1e8 (100 million pixels), so we override it here

    maxPixels=1e13,                  # Allow large exports
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()

print("Export task started. Monitor progress in the Earth Engine Tasks tab.")

Export task started. Monitor progress in the Earth Engine Tasks tab.
